## Mô hình dự đoán quản lý chi tiêu 

Thông tin thành viên
| MSSV     | Họ và Tên        | Email                   |
|----------|------------------|-------------------------|
| 22520033 | Trương Huỳnh Thúy An | 22520033@gm.uit.edu.vn |
| 22520083 | Trịnh Thị Lan Anh  | 22520083@gm.uit.edu.vn |
| 22520148 | Huỳnh Thị Hải Châu  | 22520148@gm.uit.edu.vn |
| 22520363 | Lê Văn Giáp    | 22520363@gm.uit.edu.vn |
| 22520375 | Vương Dương Thái Hà | 22520375@gm.uit.edu.vn |


### Import Libaray

In [250]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense   
from tensorflow.keras.models import load_model

In [251]:
data = pd.read_csv("data.csv")
data

,thu nhập,cố định,nhu cầu,giáo dục,thanh toán nợ,tiết kiệm,bản thân,a,b,c,d,e,f
0,31100000,7600000,0,0,2500000,1,1,0.24,0.17,0.26,0.19,0.08,0.06
1,32800000,13100000,0,0,0,1,0,0.40,0.18,0.20,0.19,0.00,0.03
2,33000000,11200000,0,1,0,1,1,0.34,0.15,0.25,0.17,0.00,0.10
3,34000000,10400000,0,1,1400000,1,1,0.31,0.20,0.24,0.18,0.04,0.03
4,31000000,6500000,0,1,3800000,1,1,0.21,0.17,0.31,0.16,0.12,0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2664,29330000,15640000,2,1,0,0,1,0.53,0.06,0.18,0.22,0.00,0.01
2665,26820000,16180000,2,0,0,0,1,0.60,0.07,0.15,0.15,0.00,0.02
2666,27730000,16530000,2,0,0,0,1,0.60,0.04,0.18,0.17,0.00,0.01
2667,28950000,16910000,2,0,0,0,1,0.58,0.03,0.16,0.16,0.00,0.07


In [252]:
data.isnull().sum()

thu nhập         0
cố định          0
nhu cầu          0
giáo dục         0
thanh toán nợ    0
tiết kiệm        0
bản thân         0
a                0
b                0
c                0
d                0
e                0
f                0
dtype: int64

### Split Data

In [253]:
X = np.array(data)[:,0:7]
Y = np.array(data)[:,7:13]
print(X)
print(Y)

[[3.110e+07 7.600e+06 0.000e+00 ... 2.500e+06 1.000e+00 1.000e+00]
 [3.280e+07 1.310e+07 0.000e+00 ... 0.000e+00 1.000e+00 0.000e+00]
 [3.300e+07 1.120e+07 0.000e+00 ... 0.000e+00 1.000e+00 1.000e+00]
 ...
 [2.773e+07 1.653e+07 2.000e+00 ... 0.000e+00 0.000e+00 1.000e+00]
 [2.895e+07 1.691e+07 2.000e+00 ... 0.000e+00 0.000e+00 1.000e+00]
 [2.645e+07 1.746e+07 2.000e+00 ... 0.000e+00 0.000e+00 1.000e+00]]
[[0.24437299 0.16783652 0.25818948 0.18591881 0.08038585 0.06329635]
 [0.39939024 0.17820915 0.20006306 0.1927493  0.         0.02958825]
 [0.33939394 0.14757885 0.24695972 0.17059292 0.         0.09547457]
 ...
 [0.5961053  0.03859579 0.18078764 0.17428451 0.         0.01      ]
 [0.58411054 0.02678687 0.16255465 0.15940921 0.         0.07      ]
 [0.66011342 0.0053385  0.15304779 0.16986224 0.         0.01      ]]


In [254]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)
print("Y_train: ",y_train.shape)
print("Y_test: ", y_test.shape)


X_train:  (2135, 7)
X_test:  (534, 7)
Y_train:  (2135, 6)
Y_test:  (534, 6)


### Descale Data

In [255]:
# Tính các giá trị thống kê cơ bản
mean = np.mean(X_train, axis=0)
median = np.median(X_train, axis=0)
std_dev = np.std(X_train, axis=0)
min_val = np.min(X_train, axis=0)
max_val = np.max(X_train, axis=0)

# Tạo DataFrame từ các giá trị thống kê
statics_df = pd.DataFrame({
    'Mean': mean,
    'Median': median,
    'Std Dev': std_dev,
    'Min Value': min_val,
    'Max Value': max_val
})

# Xếp chúng thành các hàng của các cột
statics_transpose = statics_df.transpose()

# In DataFrame đã xếp
print(statics_transpose)

                    0           1    2    3           4    5          6
Mean      25258135.83  9793578.45 0.99 0.57   837189.86 0.62  557377.36
Median    25450000.00  9290000.00 1.00 1.00        0.00 1.00       1.00
Std Dev    8652026.37  3379323.05 0.81 0.50  2073430.28 0.48 1195761.56
Min Value 10030000.00  4160000.00 0.00 0.00        0.00 0.00       0.00
Max Value 40000000.00 19930000.00 2.00 1.00 13000000.00 1.00 6000000.00


In [256]:
def de_scale(data_scale):
    for i in range(len(data_scale)):
        data_scale[i] = data_scale[i] * (max_val[i] - min_val[i]) + min_val[i]
    return data_scale

### Preprocessing Data

In [257]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
def ScalerData(data):
    scaler = MinMaxScaler()
    return scaler.fit_transform(data)

In [258]:
X_train = ScalerData(X_train)
X_test = ScalerData(X_test)
print(X_train)
print(X_test)

[[1.13113113e-01 1.49651237e-01 0.00000000e+00 ... 7.69230769e-08
  0.00000000e+00 0.00000000e+00]
 [5.25191859e-01 4.52758402e-01 5.00000000e-01 ... 7.69230769e-02
  1.00000000e+00 0.00000000e+00]
 [9.46613280e-01 3.51299937e-01 0.00000000e+00 ... 4.07692308e-01
  1.00000000e+00 0.00000000e+00]
 ...
 [1.22122122e-01 1.38871275e-01 5.00000000e-01 ... 7.69230769e-08
  1.00000000e+00 0.00000000e+00]
 [8.24157491e-02 1.43944198e-01 1.00000000e+00 ... 0.00000000e+00
  1.00000000e+00 0.00000000e+00]
 [9.79979980e-01 7.69816107e-01 1.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
[[7.86381842e-01 8.94039735e-02 5.00000000e-01 ... 0.00000000e+00
  1.00000000e+00 1.66666667e-07]
 [4.97663551e-01 9.13907285e-02 0.00000000e+00 ... 5.98290598e-01
  1.00000000e+00 0.00000000e+00]
 [3.92523364e-01 1.80794702e-01 0.00000000e+00 ... 5.12820513e-01
  1.00000000e+00 0.00000000e+00]
 ...
 [8.33110814e-01 5.19867550e-01 0.00000000e+00 ... 0.00000000e+00
  1.00000000e+00 1.66666667e-07]

### Define model Neural Network

In [259]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD

def define_model():
    model = Sequential()
    
    # Layer 1
    model.add(Dense(64, input_dim=7, activation='relu'))
    
    # Layer 2
    model.add(Dense(128, activation='relu'))
    
    model.add(Dropout(0.2))
    
    # Layer 3
    model.add(Dense(128, activation='relu'))
    
    model.add(Dropout(0.2))
    
    # Layer 4
    model.add(Dense(6, activation='relu'))  
    
    model.compile(optimizer="adamax", loss='mean_squared_error')
    
    return model

model = define_model()
model.summary()

c:\Users\vuong\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_52 (Dense)                │ (None, 64)             │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_53 (Dense)                │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_54 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_55 (Dense)                │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,118 (102.02 KB)

 Trainable params: 26,118 (102.02 KB)

 Non-trainable params: 0 (0.00 B)

In [260]:
from sklearn.ensemble import RandomForestRegressor
def define_model_2():
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    return model

### Train Model

In [261]:
import matplotlib.pyplot as plt

def train_and_plot_loss():
    model = define_model()
    history = model.fit(X_train, y_train, epochs=64, batch_size=1, validation_split=0.2, shuffle=True ,verbose=1)
    model.save('final_model.h5')
    
    # Lấy giá trị loss từ lịch sử huấn luyện
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    # Vẽ biểu đồ loss của cả hai tập train và validation
    epochs = range(1, len(train_loss) + 1)
    plt.plot(epochs, train_loss, 'b', label='Training loss')  # Đường liên tục màu xanh
    plt.plot(epochs, val_loss, 'r', label='Validation loss')  # Đường liên tục màu đỏ
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [262]:
import joblib
def train2():
    model2 = define_model_2()
    model2.fit(X_train, y_train)
    joblib.dump(model2, 'random_forest_model.pkl')


In [263]:
train2()

In [264]:
#train_and_plot_loss()

### Dự đoán và đánh giá theo R^2 score

In [265]:
import sys
import os
from sklearn.metrics import r2_score

# Load model
model = load_model("final_model.h5")
model2 = joblib.load('random_forest_model.pkl')

# Tính toán R-squared
predict_1 = model.predict(X_test)
r_squared_1 = r2_score(y_test, predict_1)

predict_2 = model2.predict(X_test)
r_squared_2 = r2_score(y_test, predict_2)

print("R-square Neural Network: ", np.round(r_squared_1, decimals = 2))
print("R-square Random Forest: ", np.round(r_squared_2, decimals = 2))


17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
R-square Neural Network:  0.83
R-square Random Forest:  0.82


### Đưa ra ouput

(a) Chi tiêu cố định: Tương ứng số tiền chi tiêu phục vụ cho nhu cầu chi tiêu cần thiết như tiền thuê nhà; tiền điện, nước; tiền ăn uống; xăng xe hoặc vé xe; tiền internet và điện thoại; …. Đảm bảo cho việc chi trả các khoản chi phí cần thiết thường ngày.

(b) Tiết kiệm: Tương ứng với số tiền chi tiêu phục vụ cho nhu cầu tiết kiệm và đầu tư như: tiết kiệm hưu trí, đầu tư cổ phiếu hoặc quỹ, những khoản chi tiêu lớn như mua nhà, mua xe, … 

(c) Bản thân: Tương ứng với số tiền chi tiêu phục vụ cho các hoạt động vui chơi, giải trí, thể thao, du lịch, mua sắm, đi ăn ngoài, nhà hàng, xem phim, …

(d) Giáo dục: Tương ứng với số tiền chi tiêu phục vụ cho mục tiêu giáo dục và phát triển bản thân: các khóa học; sách vở, tài liệu học tập; tham gia hội thảo, workshop; các khóa đào tạo nghề nghiệp hoặc kỹ năng mới, ...

(e) Thanh toán nợ: Tương ứng với số tiền chi tiêu phục vụ cho việc thanh toán các khoản vay nợ cần phải trả

(f) Khác: Tương ứng với số tiền chi tiêu phục vụ cho nhu cầu khác như: từ thiện, y tế, quà cáp, bảo trì, sửa chữa thiết bị, thú cưng và một số khoản chi tiêu phát sinh khác. 


In [266]:
# Tính MSE cho từng mẫu
mse_scores = []
for i in range(len(X_test)):
    mse = mean_squared_error([y_test[i]], [predict_1[i]])
    mse_scores.append(mse)

# Chuyển đổi thành mảng numpy để dễ dàng thao tác
mse_scores = np.array(mse_scores)
# Tạo output_matrix để xuất lưu giữ giá trị dự đoán
output_matrix = []
# Sắp xếp và chọn các mẫu có MSE thấp nhất
# Giả sử bạn muốn chọn 10 mẫu có MSE thấp nhất
top_n = 10
top_indices = np.argsort(mse_scores)[:top_n]

# Lưu các mẫu có MSE thấp nhất và các X_test tương ứng
top_samples = []
top_X_test = []
for idx in top_indices:
    top_samples.append({
        "Sample_index": idx,
        "True_value": y_test[idx],
        "Predicted_value": predict_1[idx],
        "MSE": mse_scores[idx]
    })
    top_X_test.append(X_test[idx])
    temp = []
    for item in de_scale(X_test[idx]):
        temp.append(item)
    for item in predict_1[idx]:
        temp.append(item *temp[0])
    output_matrix.append(temp)

df = pd.DataFrame(output_matrix, columns=["thu nhập",	"cố định",	"nhu cầu",	"giáo dục", "thanh toán nợ",
                                      	"tiết kiệm",	"bản thân",	"a",	"b",	"c",	"d",	"e",	"f"])
df['cố định'] = df['cố định'].apply(lambda x: '{:,.2f}'.format(x))
df['nhu cầu'] = df['nhu cầu'].apply(lambda x: '{:,.0f}'.format(x))
df['giáo dục'] = df['giáo dục'].apply(lambda x: '{:,.0f}'.format(x))
df['tiết kiệm'] = df['tiết kiệm'].apply(lambda x: '{:,.0f}'.format(x))
df



,thu nhập,cố định,nhu cầu,giáo dục,thanh toán nợ,tiết kiệm,bản thân,a,b,c,d,e,f
0,34098030.71,"9,851,821.19",2,1,7555555.56,0,0.00,9816565.97,2703846.85,6899779.89,6994831.22,6793504.63,862068.55
1,31397129.51,"10,896,192.05",1,1,0.00,1,0.00,11152913.29,5685352.40,6643557.46,6423954.89,0.00,1388735.13
2,33797930.57,"15,386,986.75",2,1,0.00,0,0.00,15126279.90,2303101.79,7277329.82,7249307.94,0.00,1357471.21
3,32297429.91,"11,836,125.83",1,0,0.00,1,0.00,11871173.05,5731283.89,6858504.94,6681867.10,0.00,1095644.13
4,31497162.88,"8,389,701.99",0,1,7444444.44,1,0.00,8675863.20,4415324.60,5281308.01,5382218.92,6758807.63,1065336.00
5,30496829.11,"11,940,562.91",1,0,0.00,1,0.00,12003147.09,5321340.96,6288018.65,6036703.89,0.00,926881.40
6,32497496.66,"10,687,317.88",1,1,0.00,1,0.00,11052517.43,5966306.29,6981723.76,6788848.27,0.00,1615118.54
7,31797263.02,"14,029,304.64",2,1,0.00,0,0.00,13801560.12,2239460.01,7031684.56,7038197.64,0.00,1387314.12
8,33697897.20,"11,522,814.57",1,0,0.00,1,0.00,11574647.83,6093649.06,7345259.66,7111679.21,0.00,1452433.20
9,18612863.82,"9,047,655.63",1,1,0.00,0,2000000.00,9017422.02,2281352.10,2345207.84,2478665.81,2155103.52,1286835.34
